# Instalación de Biopython en Apolo
Una introducción del servidor de EAFIT Apolo se encuentra [aquí](https://apolo-docs.readthedocs.io/en/latest/). 

Entre a Apolo por vía de putty o una terminal de linux (ya sea wsl o cygwin), previo inicio del Global Protect

In [ ]:
ssh lsalazarj@apolo.eafit.edu.co

Si es la primera vez, le pedirá cambiar el password. Tenga en cuenta que cuando escribe su password **no verá esta escritura** por lo que es fácil cometer errores o repetir caracteres. 

Una vez está en su cuenta puede observar los archivos que se encuentran en su *home*

pantallazoapolo

 Ahora vamos usar [conda] para crear un ambiente ([environment] donde instalaremos *biopython*, y las librerías que necesitamos.

In [ ]:
module load python/3.10_miniconda-23.5.2
conda create -n biopython_env
conda activate biopython_env
conda install -c conda-forge biopython

Estos comandos instalan una versión por defecto que a veces no funciona. Para especificar la versión que uno quiere (por ejemplo biopython compatible con python3.9 en vez 3.11)

In [ ]:
conda create -n biopython_py3.9 python=3.9
conda activate biopython_py3.9
conda install -c conda-forge biopython

Debemos salir y volver a entrar a apolo para chequear la instalación. Una vez realizamos el login, activamos el ambiente de biopython y comenzamos python

La siguiente vez que entran, ya instalado el ambiente, hay que cargar el ambiente y activarlo

In [ ]:

source activate biopython

In [ ]:

python

In [ ]:
import Bio
print(Bio.__version__)

quit() #para salir

## Compartir archivos entre Apolo y computador personal

Para intercambiar archivos entre su PC, puede utilizar WinSCP o abrir una terminal en su PC desde donde tenga almacenados los archivos a compartir. Abra una terminal **en su PC** y entre al directorio de los archivos que desea enviar a Apolo. Por ejemplo las secuencias de COVID descargadas previamente. Una vez en dicho directorio, el comando general funciona ``scp [archivo que quiero enviar] [directorio a donde lo voy a enviar]``

In [ ]:
scp Sars_cov_2.ASM985889v3.pep.all.fa.gz lsalazarj@apolo.eafit.edu.co:/home/laura

esto significa "save copy (scp) desde mi PC a usuario@servidor:/folder/" Ahora usted tiene copia de estos archivos en apolo. 

In [ ]:
mkdir covid19
mv Sars_cov_2.*.gz covid19 
cd covid19

Dentro del directorio donde vamos a trabajar, vamos escribir los comandos que utilizamos anteriormente para iterar sobre las secuencias 

## Escribir un script de python y ejecutar en Apolo  

In [ ]:
vi covid19_seqs.py

y se copian los comandos 

In [ ]:
# Script por Laura Salazar Jaramillo
# Muestra información de las secuencias de SARS-covid 19

# Importamos las librerías necesarias

import Bio
from Bio import SeqIO
from Bio.Seq import Seq

# Archivos de entrada (inputs) con rutas ABSOLUTAS
dna_file = "/home/lsalazarj/bicomp2025-21/covid19/Sars_cov_2.ASM985889v3.dna_rm.toplevel.fa"
prot_file = "/home/lsalazarj/bicomp2025-21/Sars_cov_2.ASM985889v3.pep.all.fa"


# Listamos el tamano del genoma
for record in SeqIO.parse(dna_file, "fasta"):
    print("La longitud en nc del genoma es:",len(record.seq))

# Listamos los id y longitudes de las proteínas
for record in SeqIO.parse(prot_file, "fasta"):
    print(record.id,len(record.seq),record.description.split()[9])

### script SLURM

Como se explicó durante la introducción de Apolo, el servidor está diseñado para correr los trabajos por vía de script. Es decir que todo lo que hicimos dentro de python lo debemos guardar en un script  y correrlo con el comando ``python script.py``. Este último comando no se escribe directamente en la terminal, sino que se guarda dentro de un script *slurm* (con extensión .sh)

In [ ]:
#!/bin/bash

#SBATCH --partition=learning                   # Partition (estudiantes)
#SBATCH --nodes=1
#SBATCH --ntasks=1                              # Number of tasks (processes)
#SBATCH --time=3:00                           # Walltime
#SBATCH --job-name=clase_python                 # Job name
#SBATCH --output=%x_%j.out                      # Stdout (%x-jobName, %j-jobId)
#SBATCH --error=%x_%j.err                       # Stderr (%x-jobName, %j-jobId)
#SBATCH --mail-type=ALL                         # Mail notification
#SBATCH --mail-user=lsalazarj@eafit.edu.co       # User Email


##### ENVIRONMENT CREATION #####
source activate biopython

##### JOB COMMANDS #### 
python /home/lsalazarj/bicomp2025-1/covid19_seqs.py

Guardamos este archivo y lo ejecutamos con el comando

In [ ]:
sbatch slurm_covid19seq.sh 

Para ver si la lista de prioridades le damos

In [ ]:
squeue

Una vez el script termina de correr debe producir dos archivos: *.err* y *.out* que nos dan el output o los errores para generarlo

Para copiar los archivos desde Apolo al computador personal entre a la terminal del computador y copie el archivo con la ruta completa desde Apolo al directorio donde está (indicaod por ".") o a otro directorio indicándole la ruta completa

In [ ]:
scp lsalazarj@apolo.eafit.edu.co:/home/laura/biopython/covid19_seqs.py .

### Ejemplo de bucles y condicionales

Vamos a utilizar los bucles y condicionales para buscar el codón de inicio de transcripción para el genoma de covid19

In [ ]:
# Define el codón de inicio
start_codon = 'AUG'

# Escanea la secuencia hasta encontrar el patrón
for i in range(len(covid19_seq)):
    if covid19_seq[i:i+3] == start_codon:
        print('The start codon starts at index', i)
        break
else:
    print('Codon not found in sequence.')

<div class="alert alert-block alert-info">
<b>Ejercicio</b> Escriba los comandos anteriores dentro de un script que busca el codón de inicio y ejecútelo vía otro script de *slurm*. Encontró el codón de inicio? </div>

Si no lo encotró probablmente es porque faltó un paso intermedio: transcribir el codón de DNA a RNA. Esto lo podemos hacer utilizando la función de biopython ``transcribe()``

In [ ]:
covid_mRNA = covid19_seq.transcribe()

<div class="alert alert-block alert-info">
<b>Ejercicio</b> Vuelva a correr el script anterior para el RNA mensajero y verifique si la posición indicada si contiene el códon de inicio </div>